In [1]:
cd ~/Dropbox/project_4

/home/ubuntu/Dropbox/project_4


In [2]:
from hwmf import utils, data
import re
import gensim
from gensim.models import doc2vec
from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
name_to_article = data.get_character_articles()

In [5]:
names, articles = list(zip(*name_to_article.items()))

In [5]:
names = []
articles = []
for name, article in name_to_article.items():
    names.append(name)
    articles.append(article)

In [6]:
title_black_list = utils.load_values('character_article_title_black_list.csv')
word_black_list = utils.load_values('character_article_word_black_list.csv')

def remove_black_listed(doc_string):
    black_list = sorted(word_black_list, key=lambda x: -len(x))
    patterns = []
    for pattern in word_black_list:
        patterns.append(r'\b' + pattern + '\b')
    patterns += word_black_list
    for pattern in patterns:
        doc_string = re.sub(pattern, '', doc_string)
    return doc_string

def to_word_list(sents):
    merged = merge_sents(sents)
    merged = remove_black_listed(merged)
    return gensim.utils.simple_preprocess(merged)


count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
counts = count_vectorizer.fit_transform(articles)
counts = counts.transpose()
corpus = matutils.Sparse2Corpus(counts)

In [ ]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())
lda = models.LdaMulticore(corpus=corpus, num_topics=20, minimum_probability=0.03, id2word=id2word, passes=10, workers=8)

In [21]:
lda.print_topics()

[(0,
  '0.004*"lantern" + 0.003*"green" + 0.002*"ring" + 0.002*"green lantern" + 0.002*"corps" + 0.002*"jordan" + 0.002*"hal" + 0.002*"swamp" + 0.002*"swamp thing" + 0.002*"lanterns"'),
 (1,
  '0.005*"avengers" + 0.003*"man" + 0.003*"hulk" + 0.003*"stark" + 0.002*"marvel" + 0.002*"iron" + 0.002*"iron man" + 0.001*"mario" + 0.001*"fantastic" + 0.001*"thanos"'),
 (2,
  '0.002*"captain" + 0.002*"america" + 0.002*"comics" + 0.002*"captain america" + 0.002*"marvel" + 0.001*"man" + 0.001*"tony" + 0.001*"character" + 0.001*"iron" + 0.001*"issue"'),
 (3,
  '0.004*"later" + 0.003*"character" + 0.003*"episode" + 0.003*"season" + 0.003*"father" + 0.002*"series" + 0.002*"relationship" + 0.002*"tells" + 0.002*"family" + 0.002*"time"'),
 (4,
  '0.001*"popov" + 0.001*"tig" + 0.001*"samcro" + 0.001*"clay" + 0.001*"kaz" + 0.001*"president" + 0.001*"chibs" + 0.001*"katniss" + 0.001*"wrestling" + 0.001*"billy"'),
 (5,
  '0.003*"star" + 0.002*"wars" + 0.002*"star wars" + 0.001*"film" + 0.001*"godzilla" + 

In [14]:
print(name_to_article)

In [ ]:
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans



count_vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(ng_train.data)

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, window=10, epochs=50)
model.build_vocab(train_corpus)

In [ ]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=50)

In [36]:
model.docvecs.most_similar('Zuko')

[('Iroh', 0.42297089099884033),
 ('Aang', 0.4034014344215393),
 ('Korra', 0.4025777280330658),
 ('Katara (Avatar: The Last Airbender)', 0.40019285678863525),
 ('Azula', 0.3617665767669678),
 ('Raksha (The Jungle Book)', 0.35100889205932617),
 ('Zaheer (The Legend of Korra)', 0.34087809920310974),
 ('Toph Beifong', 0.33944350481033325),
 ('Sokka', 0.3380088806152344),
 ('Tenzin (The Legend of Korra)', 0.32860681414604187)]

In [37]:
model.docvecs.most_similar('Aang')

[('Zuko', 0.4034014344215393),
 ('Katara (Avatar: The Last Airbender)', 0.39002037048339844),
 ('Korra', 0.35815495252609253),
 ('Zaheer (The Legend of Korra)', 0.3445892333984375),
 ('Tenzin (The Legend of Korra)', 0.33562031388282776),
 ('Sokka', 0.3217317461967468),
 ('Iroh', 0.3168046474456787),
 ('Bran Stark', 0.2853764295578003),
 ('Azula', 0.269309937953949),
 ('Johnny Smith (Dead Zone)', 0.2681092619895935)]

In [38]:
model.docvecs.most_similar('Professor Farnsworth')

[('Zoidberg', 0.31845948100090027),
 ('Gipsy Danger', 0.31159722805023193),
 ('Charlie Frost', 0.30390626192092896),
 ('Professor Bobo', 0.29978621006011963),
 ('Dick Solomon', 0.29420942068099976),
 ('Zapp Brannigan', 0.28526467084884644),
 ('Tom Paris', 0.285208523273468),
 ('Professor Hershel Layton', 0.2830590605735779),
 ('Thomas J. Whitmore', 0.2812904715538025),
 ('Kryten', 0.27740347385406494)]

In [68]:
def find_most_similar(word_list):
    inferred_vector = model.infer_vector(word_list)
#     print(len(inferred_vector))
    sims = model.docvecs.most_similar([inferred_vector], topn=10)
    return sims

In [69]:
word_list = to_word_list(name_to_article['Bender (Futurama)'])
find_most_similar(word_list)

[('Bender (Futurama)', 0.9172766208648682),
 ('Burt Gummer', 0.6373168230056763),
 ('Sammy the Shunter', 0.6062910556793213),
 ('The Smurfs (comics)', 0.5927327275276184),
 ('Karla (character)', 0.5844019055366516),
 ('Nick Carter-Killmaster', 0.5783843398094177),
 ('Mega Mindy', 0.5779253840446472),
 ('Yig', 0.5778729915618896),
 ('Jerom', 0.5753814578056335),
 ('Pluk van de Petteflet', 0.5753750801086426)]

In [18]:
for doc in train_corpus_docs:
    page_name = doc.tags[0]
    inferred_vector = model.infer_vector(train_corpus[page_name].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=5
    print('Character:', page_name, 'Most Similar:'
    rank = [docid for docid, sim in sims].index(page_name)
    print(rank)
    break

0
